# Deep Learning
## Formative assessment
### Week 7: Transformers

#### Instructions

In this notebook, you will write code to implement and train a Transformer classifier model.

Some code cells are provided you in the notebook. You should avoid editing provided code, and make sure to execute the cells in order to avoid unexpected errors. Some cells begin with the line: 

`#### GRADED CELL ####`

These cells require you to write your own code to complete them.

#### Let's get started!

We'll start by running some imports, and loading the dataset.

In [1]:
#### PACKAGE IMPORTS ####

# Run this cell to import all required packages. 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (TextVectorization, Dense, MultiHeadAttention, LayerNormalization, 
                                     Layer, Embedding, Dropout, GlobalAveragePooling1D)
import numpy as np
import matplotlib.pyplot as plt

<img src="figures/IMDb.png" title="IMDb" style="width: 550px;"/>  
  
#### The IMDb dataset

In this assignment, you will use the [IMDb dataset](https://https://www.imdb.com/interfaces/). This is a sentiment analysis dataset of movie reviews with binary labels. It contains 25,000 training examples and a further 25,000 for testing. 

* Maas, A.L.,  Daly, R.E.,  Pham, P.T.,  Huang, D.,  Ng, A.Y. & Potts, C. (2011), "Learning Word Vectors for Sentiment Analysis", _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_, 142-150.

Your goal is to build and train an encoder-only Transformer classifier model to predict the sentiment labels from the review text.

#### Load and prepare the data
For this assignment, you will load the IMDb dataset from the TensorFlow Datasets library:

In [4]:
# Run this cell to load the data and print the element_spec

import tensorflow_datasets as tfds

train_data = tfds.load("imdb_reviews", split="train", read_config=tfds.ReadConfig(try_autocache=False))
test_data = tfds.load("imdb_reviews", split="test", read_config=tfds.ReadConfig(try_autocache=False))

train_data.element_spec

{'label': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'text': TensorSpec(shape=(), dtype=tf.string, name=None)}

In [5]:
# View some samples

for example in train_data.shuffle(100).take(2):
    print(example['text'].numpy().decode("utf-8"))
    print(f"Label: {example['label'].numpy()}")
    print()

During a sleepless night, I was switching through the channels & found this embarrassment of a movie. What were they thinking?<br /><br />If this is life after "Remote Control" for Kari (Wuhrer) Salin, no wonder she's gone nowhere.<br /><br />And why did David Keith take this role? It's pathetic!<br /><br />Anyway, I turned on the movie near the end, so I didn't get much of the plot. But this must've been the best part. This nerdy college kid brings home this dominatrix-ish girl...this scene is straight out of the comic books -- or the cheap porn movies. She calls the mother anal retentive and kisses the father "Oh, I didn't expect tongue!" Great lines!<br /><br />After this, I had to see how it ended..<br /><br />Well, of course, this bitch from hell has a helluva past, so the SWAT team is upstairs. And yes...they surround her! And YES YES! The kid blows her brains out!!!! AHAHHAHAHAHA!!<br /><br />This is must-see TV. <br /><br />
Label: 0

"Wild Tigers I have Known." It will only be

#### Tokenizing the input sentences

We will need to convert the text into integer tokens to be able to process them in the Transformer. To do this we will use a `TextVectorization` layer and adapt it to the training data. You should now complete the following function to create and prepare this layer as follows:

* The function takes a `dataset` (a `tf.data.Dataset` object) as an argument, which has the same spec as `train_data` or `test_data` above. It also takes a `max_tokens` argument
* The `TextVectorization` should be configured to use a maximum of `max_tokens` tokens (including the masking and OOV tokens)
* It should standardize the input text by lower-casing the text and removing punctuation
* It should split the text on whitespace
* You should use the `adapt` method to compute the vocabulary using `dataset`

In [6]:
#### GRADED CELL ####

# Complete the following function.
# Make sure not to change the function name or arguments.

def configure_textvectorization(dataset, max_tokens):
    """
    This function should create a TextVectorization layer and configure it as above.
    The function should then return the TextVectorization layer.
    """
    tv = TextVectorization(max_tokens=max_tokens
                           ,standardize='lower_and_strip_punctuation', #default
                           split = "whitespace"#default 
                           )
    tv.adapt(dataset.map(lambda x: x["text"]))
    return tv
    

In [7]:
# Use your function to create and configure the TextVectorization layer

MAX_TOKENS = 20000
text_vectorization = configure_textvectorization(train_data, max_tokens=MAX_TOKENS)

In [8]:
# Test your TextVectorization layer

for example in train_data.shuffle(100).take(1):
    print(example["text"])
    print(text_vectorization(example['text']))

tf.Tensor(b'The Three Stooges has always been some of the many actors that I have loved. I love just about every one of the shorts that they have made. I love all six of the Stooges (Curly, Shemp, Moe, Larry, Joe, and Curly Joe)! All of the shorts are hilarious and also star many other great actors and actresses which a lot of them was in many of the shorts! In My opinion The Three Stooges is some of the greatest actors ever and is the all time funniest comedy team! <br /><br />One of My favorite Stooges shorts with Shemp is none other than Husbands Beware! All appearing in this short are the beautiful Christine McIntyre, Dee Green, Doris Houck, Alyn Lockwood, Johnny Kascier, Nancy Saunders, Lu Leonard, Maxine Gates, and Emil Sitka. Green and McIntyre provide great performances here! There are so many funny parts here. This is a very hilarious short. There is another similar Three Stooges short like this one called Brideless Groom and I recommend both!', shape=(), dtype=string)
tf.Tens

In [9]:
for example in train_data.take(1):
    print(example[0])

KeyError: 0

#### Preprocess the datasets

You should now complete the following function `preprocess_dataset` which you will use to preprocess the `train_data` and `test_data` Dataset objects according to the following spec:

* The function takes `dataset`, `text_vectorization_layer`, `max_seq_len`, `batch_size` and `shuffle_buffer_size` as arguments
    * `dataset` is a `tf.data.Dataset` object with the same spec as `train_data` or `test_data` above
* The `text_vectorization_layer` should be used to convert the text into integer tokens
* The maximum length of the token sequences should be `max_seq_len`. Any token sequences longer than this should be truncated
* The Datasets should return a tuple of `(tokens, label)` Tensors
* The Datasets should be shuffled with buffer size `shuffle_buffer_size`, and then batched with `batch_size`. Note that the sequences will not be the same length, so the batches should be padded with zero masking tokens where necessary (see [the docs](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#padded_batch))
* The function should then return the preprocessed `dataset` Dataset object

In [10]:
#### GRADED CELL ####

# Complete the following function.
# Make sure not to change the function name or arguments.

def preprocess_dataset(dataset, text_vectorization_layer, max_seq_len, batch_size, shuffle_buffer_size):
    """
    This function should preprocess the Dataset object as above.
    The function should then return the preprocessed Dataset.
    """
    #need the usual dataset form, since currently each example is an item not tuple
    def input_target(example):
        return example["text"], example["label"]
    def tokenize_fct(text,labels):
        return text_vectorization_layer(text), labels
    def truncate_seq(tokens, labels):
        return tokens[:max_seq_len], labels
    dataset =  dataset.map(input_target).map(tokenize_fct).map(truncate_seq)
    dataset = dataset.shuffle(shuffle_buffer_size).padded_batch(batch_size)
    return dataset
    
    

In [11]:
# Run your function to preprocess the Datasets

MAX_SEQ_LEN = 200
train_data = preprocess_dataset(train_data, text_vectorization, MAX_SEQ_LEN, 
                                batch_size=32, shuffle_buffer_size=500)
test_data = preprocess_dataset(test_data, text_vectorization, MAX_SEQ_LEN, 
                               batch_size=32, shuffle_buffer_size=500)

In [12]:
# Print the element_spec

train_data.element_spec

(TensorSpec(shape=(None, None), dtype=tf.int64, name=None),
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

In [13]:
# Inspect a data minibatch

for example in train_data.take(1):
    print(example)

(<tf.Tensor: shape=(32, 200), dtype=int64, numpy=
array([[2175,    7,   34, ...,    0,    0,    0],
       [1181,   10,   39, ...,    0,    0,    0],
       [  10,   41,  183, ...,    2,   64, 3219],
       ...,
       [   9,  399,   39, ...,    0,    0,    0],
       [  10,  405,   11, ...,    0,    0,    0],
       [ 142,    2,  584, ...,    0,    0,    0]], dtype=int64)>, <tf.Tensor: shape=(32,), dtype=int64, numpy=
array([1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0], dtype=int64)>)


Note that when we pass this integer tokens Tensor through our Transformer, we will need to be careful to not use the zero padding tokens. The mechanism to handle this is masking (see [this guide](https://www.tensorflow.org/guide/keras/masking_and_padding)), and our custom layers will need to make use of this masking mechanism.

#### Transformer architecture

We will use an encoder-only Transformer classifier architecture for the task of sentiment prediction. This will consist of a single encoder block, followed by a classifier head.

<img src="figures/encoder-only_transformer.png" alt="Transformer" style="width: 250px;"/>  

#### Positional encodings and Embedding layer

You will now implement the input embedding and positional encoding stage of the Transformer. Your model will use the deterministic positional encoding scheme $\mathbf{P}\in\mathbb{R}^{n\times d_{model}}$ as in the original Transformer:

$$
\begin{align}
P_{ti} &= \left\{
\begin{array}{c}
\sin(\omega_k t)\quad\text{for }i=2k+1,\quad(\text{for some }k\in\mathbb{N}_0)\\
\cos(\omega_k t)\quad\text{for }i=2k+2\quad(\text{for some }k\in\mathbb{N}_0)
\end{array}
\right.\\
\omega_k &= \frac{1}{10000^{2k/d_{model}}},
\end{align}
$$

where $t=1,\ldots,n$ and $i=1,\ldots,d_{model}$.

You should now complete the following `positional_encodings` function to compute the positional encoding $\mathbf{P}\in\mathbb{R}^{n\times d_{model}}$.

* The function takes `seq_len` (the number of time steps) and `d_model` the embedding dimension as integer arguments
* The function should compute a 2D Tensor of shape `(seq_len, d_model)` of positional encodings according to the above equations (be careful with python's zero-indexing, the python indices are off-by-one from the mathematical indices above)
* The function should then return the Tensor of positional encodings with type `tf.float32`

In [2]:
#### GRADED CELL ####

# Complete the following function.
# Make sure not to change the function name or arguments.

def positional_encodings(seq_len, d_model):
    """
    This function should compute the positional encodings as above.
    The function should then return the Tensor of positional encodings.
    """
    max_wavelength = 10000.

    time = np.arange(seq_len)
    inx = np.arange(d_model)
    #make the grids, it is done element wise
    I, time = np.meshgrid(inx, time)
    pe_even = np.sin(time / max_wavelength**(I/d_model))
    pe_odd = np.cos(time / max_wavelength**(I/d_model))
    #put them in the right positions
    pe = np.zeros((seq_len, d_model))
    pe[:, ::2] = pe_even[:, ::2]
    pe[:, 1::2] = pe_odd[:, ::2]
    return tf.constant(pe, dtype=tf.float32)
    

In [15]:
time = np.array([1,2,3,4,5])
pos = np.array([1,2,3])
np.meshgrid(pos,time)

[array([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]]),
 array([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3],
        [4, 4, 4],
        [5, 5, 5]])]

In [6]:
# Run your function to get the positional encodings
#this does the positional encodings for the maximum length, we may only need subsets of this, since not all sequences will be the same length
D_MODEL = 32
pos_encodings = positional_encodings(10, D_MODEL)
pos_encodings

ValueError: could not broadcast input array from shape (10,16) into shape (10,15)

The positional encodings should be added to the token embeddings in the first stage of the Transformer.

You should now complete the `__init__` and `call` methods for the following custom layer `InputEmbeddings`, which builds and returns a model that converts the integer token sequence into a sequence of embeddings, and then adds positional encodings.

* The initialiser takes the following required arguments:
    * `d_model`: the dimension of the embedding vectors
    * `pos_encodings`: the Tensor of positional encodings of shape `(max_seq_len, d_model)`, as computed by the `positional_encodings` function
    * `max_tokens`: The maximum number of integer tokens used in the input (including the masking and OOV tokens)
* The custom layer should create an `Embedding` layer with the correct input and output dimensions, that is set to mask incoming zero tokens. This layer should be set as the class attribute `embedding`
* The `call` method takes a Tensor of integer tokens of shape `(batch_size, n)` as input, where `n` is the maximum sequence length in the batch
* The `call` method should use the `Embedding` lookup layer to convert the inputs to embedding vectors, and return the sum of the embedding vectors and positional encodings

_NB: The custom layer also implements the `compute_mask` method, which has been completed for you. This is a method should be implemented whenever a layer should produce a mask (see [this guide](https://www.tensorflow.org/guide/keras/masking_and_padding) for more information). Note that this method references `self.embedding`. In this instance we want to pass on the same mask that is produced by the `Embedding` layer, so the `compute_mask` method simply calls the existing method from the `Embedding` layer._

In [17]:
#### GRADED CELL ####

# Complete the following class.
# Make sure not to change the class or methods name or arguments.

class InputEmbeddings(Layer):
    """
    This custom layer should take a batch of integer tokens as input, 
    converts them into embeddings and adds the positional encodings.
    """
    
    def __init__(self, d_model, pos_encodings, max_tokens, name='input_embeddings', **kwargs):
        super().__init__(name=name, **kwargs)
        self.pos_encodings = pos_encodings
        #output in dmodel
        self.embedding = Embedding(input_dim = max_tokens, output_dim = d_model, mask_zero = True)
        
    def compute_mask(self, inputs, mask=None):
        return self.embedding.compute_mask(inputs)
        
    def call(self, inputs):
        """
        inputs is an integer Tensor of shape (batch_size, n), where n is 
        the maximum sequence length in this batch (n \le max_seq_len)
        """       
        input_embedding = self.embedding(inputs)
        #take the positional encodings we neeed
        n = inputs.shape[-1]
        return input_embedding + self.pos_encodings[:n,:]
        

In [18]:
# Create an instance of your custom layer

input_embeddings = InputEmbeddings(D_MODEL, pos_encodings, MAX_TOKENS)

In [19]:
# Test your custom layer on an input

for tokens, _ in train_data.take(1):
    h = input_embeddings(tokens)

In [21]:
# Check that our custom layer is producing a mask

h._keras_mask

<tf.Tensor: shape=(32, 200), dtype=bool, numpy=
array([[ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ..., False, False, False],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ..., False, False, False]])>

#### Encoder block

You will now implement the encoder block of the Transformer model. This block consists of a multi-head attention block with a residual connection followed by layer normalisation, and then a pointwise feedforward network with residual connection again followed by layer normalisation.

The multi-head attention block will need to account for the masking corresponding to the zero padding tokens in the input. The way the `MultiHeadAttention` layer handles this is through the `attention_mask` argument when it is called. 

The incoming mask is a boolean Tensor with shape `(batch_size, seq_len)`, where `seq_len` is the length of the sequence of embedding vectors being input to the `MultiHeadAttention` layer. The multi-head attention is performing self-attention, so the shape of the mask required by the `attention_mask` will be `(batch_size, seq_len, seq_len)`.

Before implementing the encoder block, you should complete the following function `get_attention_mask`, which takes a single argument `mask`, which is a boolean Tensor of shape `(batch_size, seq_len)`, or `None`. If `mask` is `None`, then this function should return `None`. Otherwise, the function should return a boolean Tensor of shape `(batch_size, seq_len, seq_len)` which will be used by the `MultiHeadAttention` layer.

For a single example in the batch, suppose the vector mask $\mathbf{m}\in\mathbb{R}^n$, where $n$ is the sequence length. We would like to convert this vector mask to a matrix mask $\mathbf{M}\in\mathbb{R}^{n\times n}$, where the $i,j$-th element $M_{ij}$ is given by the element $\min (i,j)$ of the vector $\mathbf{m}$.

For example, if the incoming boolean mask was as follows:

```
mask = [[True, True, False],
        [True, False, False]]
```

where the batch size is 2 and the sequence length is 3, then the mask returned by the function `get_attention_mask` should look like:

```
returned_mask = [[[True, True, False],
                  [True, True, False],
                  [False, False, False]],
                 [True, False, False],
                 [False, False, False],
                 [False, False, False]]]
```

In [26]:
mask = np.array([[True,True,False],
        [True, False, False]])
print(mask[:,:,None])
print(mask[:,None,:])
print(mask[:,:,None]&mask[:,None,:])

[[[ True]
  [ True]
  [False]]

 [[ True]
  [False]
  [False]]]
[[[ True  True False]]

 [[ True False False]]]
[[[ True  True False]
  [ True  True False]
  [False False False]]

 [[ True False False]
  [False False False]
  [False False False]]]


In [27]:
#### GRADED CELL ####

# Complete the following function.
# Make sure not to change the function name or arguments.

def get_attention_mask(mask=None):
    """
    This function should compute the attention mask as described above.
    The function should then return the boolean Tensor.
    """
    if mask is None:
        return mask
    mask1 = mask[:, :, None]
    mask2 = mask[:, None, :]
    return mask1 & mask2

    

In [29]:
# Test your mask function
input_mask = tf.constant([[True, True, False], [True, False, False]])
get_attention_mask(input_mask)

<tf.Tensor: shape=(2, 3, 3), dtype=bool, numpy=
array([[[ True,  True, False],
        [ True,  True, False],
        [False, False, False]],

       [[ True, False, False],
        [False, False, False],
        [False, False, False]]])>

You should complete the following `EncoderBlock` custom layer, that implements the encoder block. 

* The initialiser takes the following required arguments:
    * `num_heads`: the number of attention heads to use in the multi-head attention
    * `key_dim`: the key (and query and value) dimension to use in the multi-head attention
    * `d_model`: the embedding dimension of the Transformer
    * `ff_dim`: the width of the hidden layer in the feedforward network in the encoder block
* The initialiser sets `self.support_masking = True` (this has been done for you), so that the incoming boolean mask will also be included in the output of the `EncoderBlock` layer
* The `EncoderBlock` layer should create `MultiHeadAttention`, `LayerNormalization` and `Dense` layers in the initializer as required 
    * The feedforward network should have one hidden layer of size `ff_dim` with a ReLU activation
    * The output layer of the feedforward network should have size `d_model` and no activation
* The operation of the custom layer is as follows:
    * The input to the layer is a Tensor of shape `(batch_size, seq_len, d_model)`
    * The call method also takes a `mask` argument, which will be a boolean mask of shape `(batch_size, seq_len)`. The call method should use your `get_attention_mask` function to compute the attention mask
    * Pass the input through the `MultiHeadAttention` layer (performing self-attention), passing the attention mask in the `attention_mask`. Add the resulting Tensor output to the input (residual connection) and pass through a `LayerNormalization` layer
    * Pass the resulting Tensor $h$ through the feedforward network, add this to $h$ (residual connection) and pass through a `LayerNormalization` layer
    * The custom layer should then return the resulting Tensor

In [30]:
#### GRADED CELL ####

# Complete the following class.
# Make sure not to change the class or methods name or arguments.

class EncoderBlock(Layer):
    """
    This custom layer should take a Tensor of shape (batch_size, seq_len, d_model) as input.
    It should carry out the operations as described above and return the resulting Tensor.
    """
    
    def __init__(self, num_heads, key_dim, d_model, ff_dim, name='encoder_block', **kwargs):
        super().__init__(name=name, **kwargs)
        self.supports_masking = True  # This will pass on any incoming mask
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.d_model = d_model
        self.ff_dim = ff_dim
        self.multihead_attention = MultiHeadAttention(num_heads, key_dim)
        self.ff = Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(d_model)
        ])
        #one for each layernormalisation point
        self.layernorm1 = LayerNormalization()
        self.layernorm2 = LayerNormalization()
        
    #mask is passed automatically in keras calling
    def call(self, inputs, mask=None):
        """
        inputs is a Tensor of shape (batch_size, seq_len, d_model)
        """    
        attention_mask = get_attention_mask(mask)
        h = self.multihead_attention(inputs,inputs, attention_mask = attention_mask)
        h = self.layernorm1(h + inputs)
        h_ff = self.ff(h)
        return self.layernorm2(h + h_ff)
        

In [31]:
# Create an EncoderBlock instance

encoder_block = EncoderBlock(num_heads=2, key_dim=16, d_model=32, ff_dim=32)

In [32]:
# Test your layer on a dummy input

inputs = tf.random.normal((16, 200, 32))
h = encoder_block(inputs)

In [33]:
# Test your layer on the output from the input_embeddings layer

for tokens, _ in train_data.take(1):
    h = input_embeddings(tokens)
    h = encoder_block(h)

In [34]:
# Check that the mask has been propagated

h._keras_mask

<tf.Tensor: shape=(32, 200), dtype=bool, numpy=
array([[ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       ...,
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False]])>

#### Classifier head

The final stage of our Transformer model is the classifier head. This stage consists of the following layers:

* A `GlobalAveragePooling1D` layer, that takes an incoming Tensor of shape `(batch_size, seq_len, d_model)` and reduces out the time axis to produce a Tensor of shape `(batch_size, d_model)`
* A dropout layer
* A dense layer with ReLU activation
* A dropout layer
* A dense layer with a single neuron output and sigmoid activation function

The final dense layer outputs the probability of a positive sentiment label.

You should now complete the following `get_classifier_head` function, which takes the arguments `d_model`, `dropout_rate` and `units`. The function should build and return a Sequential Model object, according to the above specification, where `dropout_rate` is used in both `Dropout` layers and `units` is used to define the width of the intermediate `Dense` layer. The `d_model` input should be used to set the `input_shape` in the first layer.

Note that the [`GlobalAveragePooling1D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling1D) layer will automatically use the incoming mask when used in the `Sequential` model, see [the guide](https://www.tensorflow.org/guide/keras/masking_and_padding) here.

In [35]:
#### GRADED CELL ####

# Complete the following function.
# Make sure not to change the function name or arguments.

def get_classifier_head(d_model, dropout_rate, units):
    """
    This function should compute classifier head model as described above.
    The function should then return the Model object.
    """
    model = Sequential([
        GlobalAveragePooling1D(input_shape=(None, d_model)),
        Dropout(dropout_rate),
        Dense(units, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    return model

In [36]:
# Create an instance of the classifier head

classifier_head = get_classifier_head(D_MODEL, dropout_rate=0.1, units=20)

In [37]:
# Print the model summary

classifier_head.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 global_average_pooling1d (G  (None, 32)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 20)                660       
                                                                 
 dropout_2 (Dropout)         (None, 20)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 681
Trainable params: 681
Non-trainable params: 0
________________________________________________________

In [38]:
# Test your classifier head model on a dummy input

inputs = tf.random.normal((8, 200, D_MODEL))
classifier_head(inputs)

<tf.Tensor: shape=(8, 1), dtype=float32, numpy=
array([[0.4942952 ],
       [0.5062677 ],
       [0.517485  ],
       [0.50191146],
       [0.51002496],
       [0.510962  ],
       [0.50928396],
       [0.47465256]], dtype=float32)>

#### Build the Transformer classifier

We now have all the components to build the complete Transformer classifier. You should now complete the following function `get_transformer_classifier` to build and compile the model.

The function takes the arguments `input_embeddings_layer`, `encoder_block_layer` and `classifier_head_layer`. It should use these layers to build a `Sequential` model, and compile it with a suitable loss function and optimizer, and an accuracy metric.

In [39]:
#### GRADED CELL ####

# Complete the following function.
# Make sure not to change the function name or arguments.

def get_transformer_classifier(input_embeddings_layer, encoder_block_layer, classifier_head_layer):
    """
    This function should compute classifier head model as described above.
    The function should then return the Model object.
    """
    model = Sequential([
        input_embeddings_layer,
        encoder_block_layer,
        classifier_head_layer
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [40]:
# Get the compiled Transformer classifier model

input_embeddings = InputEmbeddings(D_MODEL, pos_encodings, MAX_TOKENS)
encoder_block = EncoderBlock(num_heads=2, key_dim=16, d_model=32, ff_dim=32)
classifier_head = get_classifier_head(D_MODEL, dropout_rate=0.1, units=20)
transformer = get_transformer_classifier(input_embeddings, encoder_block, classifier_head)

In [41]:
# Test the Transformer classifier model

for tokens, _ in train_data.take(1):
    outputs = transformer(tokens)

#### Train the model

In [42]:
history = transformer.fit(train_data, validation_data=test_data, epochs=2)

Epoch 1/2
782/782 [==============================] - 19s 22ms/step - loss: 0.5170 - accuracy: 0.7080 - val_loss: 0.3479 - val_accuracy: 0.8480
Epoch 2/2
782/782 [==============================] - 17s 22ms/step - loss: 0.2515 - accuracy: 0.9009 - val_loss: 0.3571 - val_accuracy: 0.8554


#### Test on unlabelled data

The IMDB dataset also contains a split without labels. The following cell loads this dataset split and applies a shuffle.

In [43]:
unsupervised_data = tfds.load("imdb_reviews", split="unsupervised", 
                              read_config=tfds.ReadConfig(try_autocache=False))
unsupervised_data = unsupervised_data.shuffle(1000)

Now let's take a look at some model predictions.

In [48]:
for example in unsupervised_data.take(1):
    print(example['text'].numpy().decode("utf-8"))
    tokens = text_vectorization(example['text'])
    tokens = tokens[tf.newaxis, :MAX_SEQ_LEN]  # Add dummy batch dimension and truncate
    prob = transformer(tokens).numpy().squeeze()
    print(f"\nTransformer probability of positive label: {prob}")

this movie is pretty awful. it lacks suspense and is all around boring and worthless. i connected with none of the characters and now as i think back, i can't even remember how it ends. don't waste your time w/ it. actually i think why i don't remember the end is because i fast forwarded to the end, i do that sometimes, so i can say i've seen a film but i don't have to actually sit and watch the whole thing, i usually get the gist though. it's a bad habit really, if it's getting too long or i'm just feeling too fidgety or it's boring i'll fast forward. i always feel a little guilty when i do it though. like i'm disrespecting the director, and i am, but i've got other things to do than watch a crappy film. now mind you i've only done this a few times and this film happens to be on my 'sucky' film list.

Transformer probability of positive label: 0.0049238489009439945


Congratulations on completing this week's assignment! You have now implemented and trained an encoder-only Transformer classifier model for the task of sentiment prediction.